#Construction of promoter vector pYPKa_Z_TDH3 and terminator vector pYPKa_E_TDH3

This notebook describe the construction of E. coli vectors [pYPKa_Z_TDH3](pYPKa_Z_TDH3.gb) and [pYPKa_E_TDH3](pYPKa_E_TDH3.gb)
with an insert for which PCR primers are also designed.

The insert defined below is cloned in pYPKa using the blunt restriction 
enzymes [ZraI](http://rebase.neb.com/rebase/enz/ZraI.html) and [EcoRV](http://rebase.neb.com/rebase/enz/EcoRV.html) in 
two different plasmids. The insert cloned in ZraI will be used as a promoter, while in the EcoRV site it will be used as a 
terminator.

![pYPKa_Z and pYPKa_E plasmids](pYPK_ZE.png "pYPKa_Z and pYPKa_E plasmids")

The [pydna](https://pypi.python.org/pypi/pydna/) package is imported in the code cell below. 
There is a [publication](http://www.biomedcentral.com/1471-2105/16/142) describing pydna as well as
[documentation](http://pydna.readthedocs.org/en/latest/) available online. 
Pydna is developed on [Github](https://github.com/BjornFJohansson/pydna).

In [1]:
import pydna

The vector backbone pYPKa is read from a local [file](pYPKa.gb).

In [2]:
pYPKa = pydna.read("pYPKa.gb")

Both restriction enzymes are imported from [Biopython](http://biopython.org/wiki/Main_Page)

In [3]:
from Bio.Restriction import ZraI, EcoRV

The vector is cut with both enzymes.

In [4]:
pYPKa_ZraI  = pYPKa.linearize(ZraI)
pYPKa_EcoRV = pYPKa.linearize(EcoRV)

The insert sequence is read from a local file. This sequence was parsed from the ypkpathway data file.

In [5]:
ins = pydna.read("TDH3.gb")

The insert sequence is read from a local file]. This sequence was parsed from the ypkpathway data file.

In [6]:
fp_tail = "ttaaat"
rp_tail = "taattaa"

Design primers for the terminator promoter. The primers has specific tails in order to produce 
a [SmiI](http://rebase.neb.com/rebase/enz/SmiI.html) and a [PacI](http://rebase.neb.com/rebase/enz/PacI.html) restriction site in the EcoRV cloning position.

In [7]:
fp, rp = pydna.cloning_primers(ins, fp_tail=fp_tail, rp_tail=rp_tail)

Primers are given the names below. These primers are included in the primer list in the end of the [pathway notebook](pw.ipynb) file.

In [8]:
fp.id = "TDH3fw"
rp.id = "TDH3rv"

print(fp.format("tab"))

print(rp.format("tab"))

TDH3fw	ttaaatATAAAAAACACGCTTTTTC

TDH3rv	taattaaTTTGTTTGTTTATGTGTGTT



PCR to create the insert using the newly designed primers.

In [9]:
prd = pydna.pcr(fp, rp, ins)

The PCR product has this length in bp.

In [10]:
len(prd)

711

A figure of the primers annealing on template.

In [11]:
prd.figure()

      5ATAAAAAACACGCTTTTTC...AACACACATAAACAAACAAA3
                             |||||||||||||||||||| tm 44.1 (dbd) 54.7
                            3TTGTGTGTATTTGTTTGTTTaattaat5
5ttaaatATAAAAAACACGCTTTTTC3
       ||||||||||||||||||| tm 42.7 (dbd) 55.3
      3TATTTTTTGTGCGAAAAAG...TTGTGTGTATTTGTTTGTTT5

A suggested PCR program.

In [12]:
prd.program()


Taq (rate 30 nt/s) 35 cycles             |711bp
95.0°C    |95.0°C                 |      |SantaLucia 1998
|_________|_____          72.0°C  |72.0°C|SaltC 50mM
| 03min00s|30s  \         ________|______|
|         |      \ 50.0°C/ 0min21s| 5min |
|         |       \_____/         |      |
|         |         30s           |      |4-12°C

The final vectors are:

In [13]:
pYPKa_Z_TDH3 = (pYPKa_ZraI  + prd).looped().synced(pYPKa)
pYPKa_E_TDH3 = (pYPKa_EcoRV + prd).looped().synced(pYPKa)

The final vectors with reverse inserts are created below. These vectors theoretically make up
fifty percent of the clones. The PCR strategy below is used to identify the correct clones.

In [14]:
pYPKa_Z_TDH3b = (pYPKa_ZraI  + prd.rc()).looped().synced(pYPKa)
pYPKa_E_TDH3b = (pYPKa_EcoRV + prd.rc()).looped().synced(pYPKa)

A combination of standard primers and the newly designed primers are 
used for the strategy to identify correct clones.
Standard primers are listed [here](primers.fasta).

In [15]:
p = { x.id: x for x in pydna.parse("primers.fasta") }

##Diagnostic PCR confirmation

The correct structure of pYPKa_Z_TDH3 is confirmed by PCR using standard primers
577 and 342 that are vector specific together with the TDH3fw primer specific for the insert 
in a multiplex PCR reaction with 
all three primers present.

Two PCR products are expected if the insert was cloned, the sizes depend 
on the orientation. If the vector is empty or contains another insert, only one
product is formed.

####Expected PCR products sizes from pYPKa_Z_TDH3:

pYPKa_Z_TDH3 with insert in correct orientation.

In [16]:
pydna.Anneal( (p['577'], p['342'], fp), pYPKa_Z_TDH3).products

[Dseqrecord(-1645), Dseqrecord(-1477)]

pYPKa_Z_TDH3 with insert in reverse orientation.

In [17]:
pydna.Anneal( (p['577'], p['342'], fp), pYPKa_Z_TDH3b).products

[Dseqrecord(-1645), Dseqrecord(-879)]

Empty pYPKa clone.

In [18]:
pydna.Anneal( (p['577'], p['342'], fp), pYPKa).products

[Dseqrecord(-934)]

####Expected PCR products sizes pYPKa_E_TDH3:

pYPKa_E_TDH3 with insert in correct orientation.

In [19]:
pydna.Anneal( (p['577'], p['342'], fp), pYPKa_E_TDH3).products

[Dseqrecord(-1645), Dseqrecord(-1396)]

pYPKa_E_TDH3 with insert in reverse orientation.

In [20]:
pydna.Anneal( (p['577'], p['342'], fp), pYPKa_E_TDH3b).products


[Dseqrecord(-1645), Dseqrecord(-960)]

Calculate cseguid checksums for the resulting plasmids for future reference.
Cseguid is a checksum that uniquely describes a circular double stranded 
sequence.

In [21]:
print(pYPKa_Z_TDH3.cseguid())
print(pYPKa_E_TDH3.cseguid())

Z8LFAVHm3ruuq_dov31lqpmfeqA
8zg87HoFsdPFl5Ao-Sup64AGvFs


The sequences are named based on the name of the cloned insert.

In [22]:
pYPKa_Z_TDH3.locus = "pYPKa_Z_TDH3"[:16]
pYPKa_E_TDH3.locus = "pYPKa_Z_TDH3"[:16]

Stamp sequence with cseguid checksum. This can be used to verify the 
integrity of the sequence file.

In [23]:
pYPKa_Z_TDH3.stamp()
pYPKa_E_TDH3.stamp()

cSEGUID_8zg87HoFsdPFl5Ao-Sup64AGvFs_2015-07-22T17:28:21.951573

Sequences are written to local files.

In [24]:
pYPKa_Z_TDH3.write("pYPKa_Z_TDH3.gb")
pYPKa_E_TDH3.write("pYPKa_E_TDH3.gb")

###[pYPKa_E_TDH3](pYPKa_E_TDH3.gb)

# Download [pYPKa_Z_TDH3](pYPKa_Z_TDH3.gb)

In [25]:
import pydna
reloaded = pydna.read("pYPKa_Z_TDH3.gb")
reloaded.verify_stamp()

cSEGUID_Z8LFAVHm3ruuq_dov31lqpmfeqA

# Download [pYPKa_E_TDH3](pYPKa_E_TDH3.gb)

In [26]:
import pydna
reloaded = pydna.read("pYPKa_E_TDH3.gb")
reloaded.verify_stamp()

cSEGUID_8zg87HoFsdPFl5Ao-Sup64AGvFs